Using kernel `conda_pytorch_latest_p36`

In [1]:
import sys
sys.path.append('../../../')

In [2]:
from pathlib import Path
import os
import random

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import torch

In [4]:
from deep.constants import *

In [5]:
%load_ext autoreload
%autoreload 2

## Sagemaker Prep

In [6]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

default_bucket = 'deep-experiments-sagemaker-bucket'
sess = sagemaker.Session(default_bucket=default_bucket)

role = 'AmazonSageMaker-ExecutionRole-20210519T102514'
print(
    role
)  # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = SAGEMAKER_BUCKET
prefix = "huggingface/first"  # Replace with the prefix under which you want to store the data if needed


AmazonSageMaker-ExecutionRole-20210519T102514


### Bucket upload

In [7]:
pt_mnist_model_data = 's3://sagemaker-us-east-1-961104659532/pytorch-training-2021-05-26-09-29-45-507/output/model.tar.gz'

In [8]:
from sagemaker.pytorch import PyTorchModel

hyperparameters={
    'train_batch_size': 32,
    'model_name': 'distilbert-base-uncased'
}

model = PyTorchModel(
    entry_point="inference.py",
    source_dir=str(SCRIPTS_TRAINING_PATH / 'stefano/pytorch_estimator_base'),
    role=role,
    model_data=pt_mnist_model_data,
    framework_version="1.8.1",
    py_version="py3",
)

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

# set local_mode to False if you want to deploy on a remote
# SageMaker instance

local_mode = False

if local_mode:
    instance_type = "local"
else:
    instance_type = "ml.c4.xlarge"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

-----------

In [ ]:
import random
import numpy as np

dummy_data = {"inputs": np.random.rand(16, 1, 28, 28).tolist()}

In [ ]:
res = predictor.predict(dummy_data)


In [ ]:
# predictor.delete_endpoint()